In [32]:
import numpy as np
import pandas as pd
import random

In [5]:
data1 = pd.read_csv("../data/AAPL_05222012_0930_1300_LOB_2.csv")

In [6]:
# column name
col1 = list(data1.columns.values)[2:62]
len(col1)
col2 = [i[16:] for i in col1]
new_names = list(data1.columns.values)[0:2]+ [i[:-19] for i in col2]
data1.columns = new_names

# delete times
data1 = data1.head(203350)

In [7]:
# feature 2
level = np.arange(10)+1
bid_P = "BID_PRICE"
ask_P = "ASK_PRICE"
ask_V = "BID_SIZE"
bid_V = "ASK_SIZE"

def feature2_1(dataset):
    ret = pd.DataFrame()
    for i in level:
        ret["diff_ab" + str(i)] = data1[bid_P + str(i)] - data1[ask_P + str(i)]
    return ret

def feature2_2(dataset):
    ret = pd.DataFrame()
    for i in level:
        ret["mid_ab" + str(i)] = (data1[bid_P + str(i)]+data1[ask_P + str(i)])/2
    return ret

temp = feature2_2(data1)
#type(feature2_1(data1)[1][1])

In [53]:
# labels
# Mid-price
def label_midprice(dataset,step):
    tmp =  dataset["mid_ab1"].diff(periods = step)
    Y_midprice = pd.Series(np.nan, index= range(dataset.shape[0]),name="Y_midprice")
    Y_midprice[tmp>0] = "upward"
    Y_midprice[tmp<0] = "downward"
    Y_midprice[tmp==0] = "stationary"
    Y_midprice[0:-step] =  Y_midprice[step:]
    Y_midprice[-step:] = np.nan
    return Y_midprice
  
# sum(Y_midprice=="downward"),sum(Y_midprice=="upward")
#label_midprice(temp,step)

#Bid-ask spread crossing
def label_spread(dataset, step):
    bid_t2 = pd.Series(0, index= range(dataset.shape[0]))
    bid_t2[0:-step] = dataset["BID_PRICE1"][step:]
    indc1 = bid_t2 - dataset["ASK_PRICE1"]
    
    ask_t2 = pd.Series(0, index= range(dataset.shape[0]))
    ask_t2[0:-step] = dataset["ASK_PRICE1"][step:]
    indc2 = ask_t2 - dataset["BID_PRICE1"]
    Y_spread = pd.Series("stationary", index= range(dataset.shape[0]))
    Y_spread[indc1>0] = "upward"
    Y_spread[indc2<0] = "downward"
    Y_spread[-step:] = np.nan
    return Y_spread

# sum(Y_spread=="downward"),sum(Y_spread=="upward")
# label_spread(data1,100 ) #1s
# question here

In [49]:
# merge dataset
def merge_dataset(dataset_ls):
    n = len(dataset_ls)
    ret = pd.DataFrame(index = range(data1.shape[0]))
    for i in dataset_ls:
        ret = pd.concat([ret,i],axis = 1)
    return ret

In [56]:
# randomly select given ratio
def random_subset(dataset,label,ratio,size):
    n_u,n_d,n_s = int(ratio[0]/sum(ratio)*size),int(ratio[1]/sum(ratio)*size),int(ratio[2]/sum(ratio)*size )
    ind_u = list(dataset[dataset[label]=="upward"].index)
    ind_d = list(dataset[dataset[label]=="downward"].index)
    ind_s = list(dataset[dataset[label]=="stationary"].index)
    indice = np.array([])
    indice = np.append(indice,np.random.choice(ind_u,replace=False,size = n_u))
    indice = np.append(indice,np.random.choice(ind_d,replace=False,size = n_d))
    indice = np.append(indice,np.random.choice(ind_s,replace=False,size = n_s))  
    ret = dataset.loc[np.sort(indice).astype(int),:]
    return ret


In [59]:
random_subset(hh,"Y_midprice",(1,1,2),1000).shape
a = [temp,label_midprice(temp,5)]
hh = merge_dataset(a)

(1000, 11)